# Naver-Blog-URL crawling

In [13]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import requests
import re
import pandas as pd

word = ['삼겹살 맛집']
# word = ['삼겹살 맛집', '곱창 맛집', '치킨 맛집']
dic = {}

for i in word:
    #print(i)
    n = 1
    val = []
    for num in range(1): #페이지수(한페이지에 블로그10개씩잇음)
        word_encode = urllib.parse.quote(i)
        driver = webdriver.Chrome('chromedriver85')
        driver.get('https://search.naver.com/search.naver?where=post&sm=tab_jum&query={}&start={}1'.format(word_encode, num))

        time.sleep(1)

        #print(word_encode)
        html = driver.page_source
        #print(html)
        
        bs = BeautifulSoup(html, 'html.parser') #문서 파싱 
        a = bs.findAll("a", {"class":"sh_blog_title _sp_each_url _sp_each_title"})
        count = 0
        driver.close()
        
        for af in a:
            if count < 10: #위에서 정한 페이지수*10
                #print(n)
                #print(af['href'])
                count += 1
                n += 1
                if 'blog.naver' in af['href'] or 'blog.me' in af['href']: #블로그중에 daum,tistory가 있어서 걸러주도록한다.
                    val.append(af['href'])
            else:
                break
                
    dic[i] = val
print(dic)


{'삼겹살 맛집': ['https://ellisha0706.blog.me/222036196703', 'https://blog.naver.com/musicalyoon?Redirect=Log&logNo=222055437929', 'https://blog.naver.com/woowim?Redirect=Log&logNo=222056167298', 'https://blog.naver.com/hangilchi?Redirect=Log&logNo=222018060805', 'https://kkiibbb.blog.me/222054113153', 'https://blog.naver.com/suyeon052?Redirect=Log&logNo=222060027136', 'https://blog.naver.com/yoo_h_byul?Redirect=Log&logNo=222065835684', 'https://ssomerry.blog.me/222071888269', 'https://blog.naver.com/l_lli?Redirect=Log&logNo=222075941356', 'https://rlacksdyd12.blog.me/222049780675']}


# Naver-Blog-URL을 이용하여 블로그속의 상호명 crawling
## naver는 블로그속에있는 내용 크롤링할때 iframe사용

In [14]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'} 
dataset = pd.DataFrame(columns=['맛집','주소','상호명'])
index = 0
for w in word:
    for mat in dic[w]:
        script = []
        URL = mat
        driver = webdriver.Chrome('./chromedriver85.exe')
        driver.get(URL)
        html = driver.page_source

        bs = BeautifulSoup(html, 'html.parser')
        target_URL = bs.find('iframe').get('src')
        target_URL = 'https://blog.naver.com' + target_URL
        response = requests.get(target_URL, headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        if soup.findAll('strong', {"class": "se-map-title"}):
            data =soup.find('strong', {"class": "se-map-title"})
            script.append(data.text)
            #print(data.text)
        elif soup.findAll('span', {"class":"pcol2 fil5"}):
            data =soup.find('span', {"class":"pcol2 fil5"})
            script.append(data.text)
            #print(data.text)    
        else:
            script.append(None)
            #print('none')
        driver.close()
        #for txt_line in data:
        #    script.append(txt_line)
        dataset.loc[index] = [w, target_URL, script]
        print(index) #잘 되고있는지 확인하기 위해,,ㅎ
        index+= 1
dataset

0
1
2
3
4
5
6
7
8
9


,맛집,주소,상호명
0,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=ell...,[VVERTIGO]
1,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=mus...,[맛찬들왕소금구이 포항죽도점]
2,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=woo...,[나무꾼이야기 양곡점]
3,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=han...,[제주이마이가 해운대본점]
4,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=kki...,[안양 늘보리냉삼집]
5,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=suy...,[총각집]
6,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=yoo...,[영통골목식당]
7,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=sso...,[연탄생삼겹고추장구이]
8,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=l_l...,[묵은지생삼겹살]
9,삼겹살 맛집,https://blog.naver.com/PostView.nhn?blogId=rla...,[교대이층집 여의도점]


In [34]:
# dataset.to_csv('list1.csv', encoding='utf-8-sig')